In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.inspection import permutation_importance
import seaborn as sns
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import random
from tqdm import tqdm
import copy
import torch
import datetime

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [2]:
def wrmse(target, pred, a):
    w = 1/(a*target + np.finfo(float).eps)
    wrmse = np.sqrt((w*(target-pred)**2).sum()/len(target))
    return wrmse

## Чтение данных
Данные предобработаны ноутбуком Split_data_0.ipynb


In [3]:

# Номера агрегатов
agg_l = [4,5,6,7,8,9]
## В дальнейшим будет идти работа со словарем data, ключ data - номер агрегата
data = {}
for agg_num in agg_l:
    data[agg_num] = {}
    df_x = pd.read_parquet(f'dataset_splited/x_train_{agg_num}.parquet')
    df_x_test = pd.read_parquet(f'dataset_splited/x_test_{agg_num}.parquet')
    df_y = pd.read_parquet(f'dataset_splited/y_train_{agg_num}.parquet').astype('uint8')
    data[agg_num]['x_train'] = df_x
    data[agg_num]['x_test_submit'] = df_x_test
    data[agg_num]['y_train'] = df_y


In [4]:
df_m = pd.read_excel('Датасеты/messages.xlsx')
df_int = pd.read_excel('Датасеты/test_intervals.xlsx')

df_m['ОПИСАНИЕ'] = df_m['ОПИСАНИЕ'].str.lower().str.strip('.')
df_m.loc[df_m['ОПИСАНИЕ'].str.contains('масл')&df_m['ОПИСАНИЕ'].str.contains('теч'), 'ОПИСАНИЕ'] =\
        'утечка масла'

df_m.loc[df_m['ОПИСАНИЕ'].str.contains('масл')&df_m['ОПИСАНИЕ'].str.contains('теч'), 'ОПИСАНИЕ'] =\
        'утечка масла'
df_m.loc[df_m['ОПИСАНИЕ'].str.contains('прос'), 'ОПИСАНИЕ'] = 'просос'

In [5]:
# Приведение колонок из разных агрегатов к одному виду 
def proc_columns_x(df_x):
    x_cols = pd.DataFrame(df_x.columns, columns = ['col'])
    x_cols.loc[x_cols['col']!='target', 'num'] = x_cols.loc[x_cols['col']!='target', \
                                                            'col'].str.extract('ЭКСГАУСТЕР (\d)*').astype(int).values
    x_cols['col_n'] = x_cols['col'].str.replace('ЭКСГАУСТЕР (\d).', '', regex=True).str.strip(' ')
    df_x['agg_num'] = x_cols['num'].mean().astype(int)
    df_x.rename(columns = {i['col']:i['col_n'] for m, i in x_cols.iterrows()}, inplace=True)
    
    
for agg_num in agg_l:
    proc_columns_x(data[agg_num]['x_train'])
    proc_columns_x(data[agg_num]['x_test_submit'])
    data[agg_num]['y_raw_columns'] = data[agg_num]['y_train'].columns
    data[agg_num]['y_train'].columns = data[agg_num]['y_train'].columns\
                                            .str.replace('Y_ЭКСГАУСТЕР А/М №(\d)_', '', regex=True)\
                                            .str.replace('ЭКСГ.№(\d)', '', regex=True)\
                                            .str.replace('ЭКСГ. №(\d)', '', regex=True)\
                                            .str.replace('ЭКСГ.  №(\d)', '', regex=True)\
                                            .str.replace('А/М №(\d)', '', regex=True)\
                                            .str.replace('А/М№(\d)', '', regex=True)\
                                            .str.replace('ЭКСГАУСТЕРА №(\d)', '', regex=True)\
                                            .str.replace('ЭКСГАУСТЕРА№(\d)', '', regex=True)\
                                            .str.replace('ЭКСГАУСТЕРА №(\d)', '', regex=True)\
                                            .str.replace('ВОЗБУДИТЕЛЬ', 'ВЗБ', regex=True)\
                                            .str.replace('ТИРИСТ.', 'ВЗБ', regex=True)\
                                            .str.strip()
    data[agg_num]['y_proc_columns'] = data[agg_num]['y_train'].columns

In [6]:
# Пример датасета для 4 агрегата
# Все агрегаты имеют одинаковую структуру
data[agg_num]['x_train'].head()

,ТОК РОТОРА 1,ТОК РОТОРА 2,ТОК СТАТОРА,ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ,ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1,ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2,ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3,ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4,ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ,ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ,ВИБРАЦИЯ НА ОПОРЕ 1,ВИБРАЦИЯ НА ОПОРЕ 2,ВИБРАЦИЯ НА ОПОРЕ 3,ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.,ВИБРАЦИЯ НА ОПОРЕ 4,ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.,agg_num
DT,,,,,,,,,,,,,,,,,
2019-01-16 13:21:00,2.470000,2.470000,0.170000,0.0,20.280000,17.040000,17.040000,15.410000,6.085000,12.170000,0.040000,0.060000,0.080000,0.100000,0.060000,0.080000,9
2019-01-16 13:21:10,2.465278,2.465278,0.173611,0.0,20.038275,17.036613,16.063092,13.790000,5.192110,12.574638,0.033854,0.065972,0.072917,0.086806,0.294271,0.093750,9
2019-01-16 13:21:20,2.328318,2.328318,0.173611,0.0,19.650670,17.036613,16.135206,15.143644,5.680000,11.360000,0.038580,0.068480,0.069444,0.084877,0.265239,0.088735,9
2019-01-16 13:21:30,2.424190,2.424190,0.173611,0.0,19.876022,16.955486,16.063090,14.440548,5.759994,12.574638,0.039931,0.060764,0.067708,0.092882,1.231771,0.078993,9
2019-01-16 13:21:40,2.424190,2.424190,0.190972,0.0,20.119401,16.712107,16.230000,14.846180,4.870000,12.980000,0.035590,0.058160,0.072049,0.087674,1.410000,0.085937,9


## Обрезка старых данных

In [7]:
cut_old_data = '2020-01-01 00:00:00'
for agg_num in agg_l:
    # Удаление старых данных - распределение скорее всего неактуально
    data[agg_num]['x_train'] = data[agg_num]['x_train'].loc[cut_old_data:]
    data[agg_num]['y_train'] = data[agg_num]['y_train'].loc[cut_old_data:]


## Вычленение периодов M1

In [8]:
def get_num_of_downtime(i, df_y):
    df_f = df_y[df_y[i]==1]
    if len(df_f)==0: return 0
    df_dt_diff = pd.Series(df_f.index).diff().value_counts()
    if len(df_dt_diff)==1: return 1
    num_of_downtime = len(df_dt_diff)
    return num_of_downtime

y_vals = {}

for agg_num in agg_l:
    df_y_vals = pd.DataFrame()
    for i in data[agg_num]['y_train'].columns:
        dwn = get_num_of_downtime(i, data[agg_num]['y_train'])
        y_val = pd.DataFrame(data[agg_num]['y_train'][i].value_counts()).T
        y_val.index = [y_val.columns.name]
        y_val['num_of_downtimes'] = dwn
        df_y_vals = pd.concat([df_y_vals, y_val])
    df_y_vals['agg_num'] = agg_num
    df_y_vals = df_y_vals.fillna(0)
    df_y_vals = df_y_vals[['agg_num', 0,2,1,'num_of_downtimes']]
    y_vals[agg_num] = df_y_vals
    print(f'Agg num: {agg_num}')
    cols_with_downtime = df_y_vals[df_y_vals[1]!=0].index.values
    data[agg_num]['y_train'] = data[agg_num]['y_train'][cols_with_downtime]
    display(df_y_vals[df_y_vals[1]>1])

Agg num: 4


,agg_num,0,2,1,num_of_downtimes
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4,4,5655650.0,659787.0,403.0,1
РЕДУКТОР ГАЗ. ЗАДВИЖКИ,4,1938191.0,4377072.0,577.0,1
ЭЛЕКТРОАППАРАТУРА,4,5033276.0,1282522.0,42.0,1
ЗАДВИЖКА,4,5104914.0,1210479.0,447.0,3
РОТОР,4,51368.0,6262619.0,1853.0,5
ПОДШИПНИК ОПОРНО-УПОРНЫЙ,4,1169088.0,5146193.0,559.0,1
ЭКСГАУСТЕР,4,6314456.0,0.0,1384.0,2


Agg num: 5


,agg_num,0,2,1,num_of_downtimes
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4,5,1916802.0,4398778.0,260.0,3
ЗАДВИЖКА,5,4318611.0,1997025.0,204.0,1
ПОДШИПНИК ОПОРНЫЙ №1,5,3966701.0,2348904.0,235.0,5
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ1,5,6313741.0,1897.0,202.0,4
ЭКСГАУСТЕР,5,6315677.0,0.0,163.0,2
РОТОР,5,3704297.0,2611284.0,259.0,4
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА,5,6315359.0,0.0,481.0,1
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ2,5,6315732.0,0.0,108.0,1


Agg num: 6


...

,agg_num,0,2,1,num_of_downtimes
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4,7,0.0,6314271.0,1569.0,3
КОРПУС,7,2417104.0,3896234.0,2502.0,2
РОТОР,7,0.0,6313177.0,2663.0,4
ЭКСГАУСТЕР,7,6311704.0,0.0,4136.0,3
ЭЛЕКТРООБОРУДОВАНИЯ,7,6289182.0,26610.0,48.0,1


Agg num: 8


,agg_num,0,2,1,num_of_downtimes
РОТОР,8,0.0,6311556.0,4284.0,8
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4,8,4947961.0,1356798.0,11081.0,2
ВЗБРНЫЙ ВЗБ ТВ-400 ЭКСГ8 ВУ1,8,6315611.0,0.0,229.0,2
ЭЛЕКТРООБОРУДОВАНИЯ,8,6301841.0,13921.0,78.0,1


Agg num: 9


,agg_num,0,2,1,num_of_downtimes
ЭЛЕКТРООБОРУДОВАНИЯ,9,5493821.0,821983.0,36.0,1
ТР-Р ТМ-6300-10/6,9,1894945.0,4420456.0,439.0,1


In [9]:
for agg_num in agg_l:
    print(f"Количество M1 простоев для агрегата {agg_num}: {y_vals[agg_num]['num_of_downtimes'].sum()}")

Количество M1 простоев для агрегата 4: 14
Количество M1 простоев для агрегата 5: 21
Количество M1 простоев для агрегата 6: 6
Количество M1 простоев для агрегата 7: 13
Количество M1 простоев для агрегата 8: 13
Количество M1 простоев для агрегата 9: 2


In [10]:
downtime_cols = pd.concat([y_vals[agg_num][y_vals[agg_num][1]!=0] for agg_num in agg_l]).reset_index(names=['feature'])
print(f'Всего количество простоев M1:')
display(downtime_cols['feature'].value_counts())
downtime_cols = downtime_cols.groupby(['feature', 'agg_num'])['num_of_downtimes'].sum().unstack().fillna(0).astype(int)
downtime_cols['sum_'] = downtime_cols.sum(axis=1)
display(downtime_cols)
display(downtime_cols[downtime_cols['sum_']>1])


Всего количество простоев M1:


feature
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4    4
РОТОР                             4
ЭКСГАУСТЕР                        4
ЗАДВИЖКА                          3
ЭЛЕКТРООБОРУДОВАНИЯ               3
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА      2
РЕДУКТОР ГАЗ. ЗАДВИЖКИ            1
ЭЛЕКТРОАППАРАТУРА                 1
ПОДШИПНИК ОПОРНО-УПОРНЫЙ          1
ПОДШИПНИК ОПОРНЫЙ №1              1
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ1      1
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ2      1
КОРПУС                            1
ВЗБРНЫЙ ВЗБ ТВ-400 ЭКСГ8 ВУ1      1
ТР-Р ТМ-6300-10/6                 1
Name: count, dtype: int64

agg_num,4,5,6,7,8,9,sum_
feature,,,,,,,
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ1,0,4,0,0,0,0,4
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ2,0,1,0,0,0,0,1
ВЗБРНЫЙ ВЗБ ТВ-400 ЭКСГ8 ВУ1,0,0,0,0,2,0,2
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА,0,1,1,0,0,0,2
ЗАДВИЖКА,3,1,4,0,0,0,8
КОРПУС,0,0,0,2,0,0,2
ПОДШИПНИК ОПОРНО-УПОРНЫЙ,1,0,0,0,0,0,1
ПОДШИПНИК ОПОРНЫЙ №1,0,5,0,0,0,0,5
РЕДУКТОР ГАЗ. ЗАДВИЖКИ,1,0,0,0,0,0,1


agg_num,4,5,6,7,8,9,sum_
feature,,,,,,,
ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ1,0,4,0,0,0,0,4
ВЗБРНЫЙ ВЗБ ТВ-400 ЭКСГ8 ВУ1,0,0,0,0,2,0,2
ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА,0,1,1,0,0,0,2
ЗАДВИЖКА,3,1,4,0,0,0,8
КОРПУС,0,0,0,2,0,0,2
ПОДШИПНИК ОПОРНЫЙ №1,0,5,0,0,0,0,5
РОТОР,5,4,0,4,8,0,21
ЭКСГАУСТЕР,2,2,1,3,0,0,8
ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4,1,3,0,3,2,0,9


In [11]:
print('Всего простоев:', downtime_cols['sum_'].sum())

Всего простоев: 69


In [12]:
# Признаки выше можно сегментировать следующим образом:

# Возбудители
### - ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ1
### - ВЗБ ВЗБ ВТ-РЭМ-400 ЭКСГ5 ВУ2
### - ВЗБРНЫЙ ВЗБ СПВД-М10-400-5
### - ВЗБРНЫЙ ВЗБ ТВ-400 ЭКСГ8 ВУ1



# ПОДШИПНИКИ
### - ПОДШИПНИК ОПОРНО-УПОРНЫЙ
### - ПОДШИПНИК ОПОРНЫЙ №1
### - ПОДШИПНИК ОПОРНЫЙ №2



# Электричество
### - РОТОР
### - ТР-Р ТМ-6300-10/6
### - ЭЛЕКТРОАППАРАТУРА
### - ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4
### - ЭЛЕКТРООБОРУДОВАНИЯ
### - ЗАДВИЖКА


# Другое
### - КОРПУС
### - ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА
### - РЕДУКТОР ГАЗ. ЗАДВИЖКИ
### - ЭКСГАУСТЕР



In [13]:
def get_downtimes(df, col, type_):
    ok_types = [0,1,2]
    ok_types.remove(type_)
    st_dates = df[(df[col]==type_) & (df[col].shift(1).isin(ok_types))].index
    end_dates = df[(df[col].shift(1)==type_) & (df[col].isin(ok_types))].index
    dates = list(zip(st_dates, end_dates))
    return dates



def get_all_downtimes(df_y, type_):
    type_ = type_.lower()
    type_ = {'m3':2, 'm1':1}[type_]
    y_cols = [i for i in df_y.columns if (df_y[i]==1).sum()!=0]
    downtimes = []
    for i in y_cols:
        downtimes += get_downtimes(df_y, i, type_)
    return downtimes


for agg_num in agg_l:
    data[agg_num]['downtimes'] = sorted(get_all_downtimes(data[agg_num]['y_train'], type_ = 'M1'))
    data[agg_num]['downtimes'] = \
                pd.DataFrame(data[agg_num]['downtimes'], columns = ['start_downtime', 'end_downtime'])

# Генерация фичей

### Подготовка датасета

In [14]:
# Удаление ненужных колонок
# ТОК РОТОРА 2 - дубликат ТОК РОТОРА 1
cols_to_drop = ['ТОК РОТОРА 2']
for agg_num in agg_l:
    data[agg_num]['x_train'].drop(columns = cols_to_drop, inplace=True)
    data[agg_num]['x_test_submit'].drop(columns = cols_to_drop, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_27976\3291706359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[agg_num]['x_train'].drop(columns = cols_to_drop, inplace=True)


In [15]:
# квантиля каждого показателя для генерации статистик аномальности
for agg_num in agg_l:
    quantiles = data[agg_num]['x_train'].quantile([0.025, 0.98])
    quantiles.index = ['min', 'max']
    data[agg_num]['thresholds'] = quantiles.T.drop(['agg_num'])

### Генерация фичей для временных рядов

In [16]:
## Т.к. многие фичи имеют схожую природу и корреляции, можно агрегиовать показали их аномальности

cols_groups = {'amp':['ТОК РОТОРА 1', 'ТОК СТАТОРА'],
               'oil_temp':[ 'ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ', 'ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ', 'ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ'],\
                'temp_bearing':['ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1', 'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2',\
                              'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4'],\
                'vibr': ['ВИБРАЦИЯ НА ОПОРЕ 2', 'ВИБРАЦИЯ НА ОПОРЕ 3',\
                         'ВИБРАЦИЯ НА ОПОРЕ 4', 'ВИБРАЦИЯ НА ОПОРЕ 1',\
                         'ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 'ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.']}

cols = pd.DataFrame(data[4]['thresholds']['min']).index.values

In [17]:
# Функция для генерации фичей - агрегирующих статистик временного ряда

def get_group_stats(df, split_by_group=True):
    
    if split_by_group:
        df_g = pd.DataFrame(columns = cols_groups.keys())
        for k,v in cols_groups.items():
            if split_by_group:
                df_g[k] = df[v].sum(axis=1)
    else:
        df_g = pd.DataFrame()
        df_g['anomaly_score'] = df[cols].sum(axis=1)

    return df_g

def generate_features(agg_num, table = 'x_train'):
    # используется несколько периода временной агрегации 
    win_sizes = ['2h']
    x = pd.DataFrame()
    cols = pd.DataFrame(data[agg_num]['thresholds']['min']).index.values
    for n, ws in enumerate(win_sizes):
        # Сколько фичей за выбранный период агрегации были меньше чем квантиль 0.025
        min_score = (data[agg_num][table][cols]<data[agg_num]['thresholds']['min']).rolling(ws).sum()\
                        .fillna(method='ffill')
        min_score = get_group_stats(min_score, split_by_group=False)
        # Сколько фичей за выбранный период агрегации были Больше чем квантиль 0.98 
        max_score = (data[agg_num][table][cols]>data[agg_num]['thresholds']['max']).rolling(ws).sum()\
                        .fillna(method='ffill')
        max_score = get_group_stats(max_score, split_by_group=False)
        # Объеденение min и max
        quantile_outlier = min_score + max_score
        quantile_outlier.columns = [f'{i}_{ws}_quantile_stats' for i in quantile_outlier.columns]
                        
        # Std показателя за выбранный период
        ### Т.к. внутри каждой группы рахные показатели имею 1 единицу измерения и схожее распределение - 
        ### можно попровать их объеденить
        std_score = data[agg_num][table][cols].rolling(ws).std()
        std_score = get_group_stats(std_score)
        std_score.columns = [f'{i}_{ws}_std_stats' for i in std_score.columns]
        
        if n==0:
            x = pd.concat([quantile_outlier, std_score], axis=1)
        else:
            x = pd.concat([x, quantile_outlier, std_score], axis=1)
        
        del quantile_outlier, std_score
    
    # Сколько фичей принимали значения nan
    nan_score = data[agg_num][table].isna().sum(axis=1)
    data[agg_num][table] = x
    data[agg_num][table]['nan_score'] = nan_score
    

In [18]:
# Добавление в тест данные из трейна дл расчета агрегирующих статистик.
# В дальнейшим будет удалено
test_start_dt = data[agg_num]['x_test_submit'].index[0]
for agg_num in tqdm(agg_l):
    data_from_train = data[agg_num]['x_train'][test_start_dt-pd.Timedelta(days=1):]
    data[agg_num]['x_test_submit'] = pd.concat([data_from_train, data[agg_num]['x_test_submit']])

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  9.62it/s]


In [19]:
# Генерация признаков для каждого агрегата
for agg_num in tqdm(agg_l):
    generate_features(agg_num, 'x_train')
    generate_features(agg_num, 'x_test_submit')

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:39<00:00, 16.59s/it]


In [20]:
## Удаление данных из трейна в тесте
for agg_num in tqdm(agg_l):
    data[agg_num]['x_test_submit'] = data[agg_num]['x_test_submit'].loc[test_start_dt:]

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 3829.25it/s]


## Генерация таргета

In [21]:
for agg_num in agg_l:
    data[agg_num]['y_train']['m1'] =  (data[agg_num]['y_train']==1).any(axis=1)
    tm_m1 = data[agg_num]['y_train'][data[agg_num]['y_train']['m1']].apply(lambda x: x[x==1].index.values[0], axis=1)
    data[agg_num]['x_train']['is_downtime'] = data[agg_num]['y_train']['m1']
    data[agg_num]['x_train']['time_to_downtime'] = None
    data[agg_num]['x_train']['next_downtime_start_time'] = None
    data[agg_num]['x_train']['downtime_tm'] = tm_m1
    
    data[agg_num]['x_train'].loc[data[agg_num]['downtimes']['start_downtime'].values, 'next_downtime_start_time'] =\
            data[agg_num]['x_train'].loc[data[agg_num]['downtimes']['start_downtime'].values].index

    data[agg_num]['x_train']['next_downtime_start_time'] = \
                data[agg_num]['x_train']['next_downtime_start_time'].fillna(method='bfill')
    data[agg_num]['x_train']['downtime_tm'] = data[agg_num]['x_train']['downtime_tm'].fillna(method='bfill')

    data[agg_num]['x_train']['time_to_downtime'] = (data[agg_num]['x_train']['next_downtime_start_time'] - \
                                                                data[agg_num]['x_train'].index)/pd.Timedelta(seconds=1)
    

In [22]:
for agg_num in agg_l:

    data[agg_num]['x_train'][ 'target_cls'] = None

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=70) & \
                             (data[agg_num]['x_train']['time_to_downtime']>10), 'target_cls'] = '1m'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=400) & \
                             (data[agg_num]['x_train']['time_to_downtime']>80), 'target_cls'] = '5m'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=2600) & \
                             (data[agg_num]['x_train']['time_to_downtime']>600), 'target_cls'] = '30m'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=4800) & \
                             (data[agg_num]['x_train']['time_to_downtime']>3000), 'target_cls'] = '1h'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=11800) & \
                             (data[agg_num]['x_train']['time_to_downtime']>5400), 'target_cls'] = '3h'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=30000) & \
                             (data[agg_num]['x_train']['time_to_downtime']>15000), 'target_cls'] = '6h'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']<=80000) & \
                             (data[agg_num]['x_train']['time_to_downtime']>40000), 'target_cls'] = '12h'

    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']>100000) & \
                                 (data[agg_num]['x_train']['time_to_downtime']<=300000), 'target_cls'] = '1d'
    
    data[agg_num]['x_train'].loc[(data[agg_num]['x_train']['time_to_downtime']>1000000) & \
                             (data[agg_num]['x_train']['time_to_downtime']<=3000000), 'target_cls'] = '1M'
    
    
low_vals = {'0':0, '1m':10, '5m':80, '30m':600, '1h':3000, \
            '3h':5400, '6h':15000, '12h':40000, '1d':100000, '1M':1000000}

dur_vals = {'1m':60, '5m':300, '30m':400, '1h':900, \
            '3h':2000, '6h':6000, '12h':10000, '1d':50000, '1M':100000}

# Предсказание периодов по статистикам аутлаеров

## Task 3

In [23]:
# Расчет априорного распределения M1
expected_nums_of_downtown = {}
for agg_num in agg_l:
    expected_nums_of_downtown[agg_num] = \
            (y_vals[agg_num][y_vals[agg_num]['num_of_downtimes']!=0]['num_of_downtimes']/2).apply(np.ceil).astype(int)

In [24]:
# Выбор колонок для статистик
x_cols = list(data[agg_num]['x_train'].columns)

to_remove = ['is_downtime', 'time_to_downtime', 'next_downtime_start_time', 'downtime_tm', 'target_cls']
for r in to_remove:x_cols.remove(r)

In [137]:
# l = []
# for agg_num in agg_l:
#     l += list((data[agg_num]['downtimes']['end_downtime'] - \
#           data[agg_num]['downtimes']['start_downtime'])/pd.Timedelta(seconds=1))
# median_time_of_downtime = np.median(l)

median_time_of_downtime = 960
inf_const = 60*60*24*30


In [238]:
# Поиск целевых событий методом look-a-like
threshs = {}
for agg_num in agg_l:
    means = data[agg_num]['x_train'][x_cols].mean()
    std = data[agg_num]['x_train'][x_cols].std()
    stats = data[agg_num]['x_train'].groupby(['target_cls', 'downtime_tm'])[x_cols].mean()
    data[agg_num]['means'] = means
    data[agg_num]['std'] = std
    stats = (stats - means)/std
    x_train = data[agg_num]['x_train'][x_cols].copy()
    x_test = data[agg_num]['x_test_submit'][x_cols].copy()
    x_train = (x_train - means)/std
    x_test = (x_test - means)/std

    for n, i in stats.loc[list(dur_vals.keys())[::-1]].iterrows():
        exp_dwnt = expected_nums_of_downtown[agg_num]
        x_test[n] = ((x_test[x_cols]-i)**2).mean(axis=1)
        if n[1] not in x_test.columns:
            x_test[n[1]] = None
        for d_n in range(exp_dwnt[n[1]]):
            threshs[(agg_num, ) + n] = x_test[n].min()
            dt_downtime = x_test.iloc[x_test[n].argmin()].name
            x_test.loc[dt_downtime-pd.Timedelta(hours=2):dt_downtime-pd.Timedelta(hours=3), n] = x_test[n].max()
            x_test.loc[dt_downtime: dt_downtime+pd.Timedelta(seconds = dur_vals[n[0]]), n[1]] = n[0]
            if n[0]=='1m':
                x_test.loc[dt_downtime+pd.Timedelta(seconds = dur_vals[n[0]]):\
                        dt_downtime+pd.Timedelta(seconds = dur_vals[n[0]]+median_time_of_downtime), \
                                                                    n[1]] = '0'
                
    for i in stats.reset_index()['downtime_tm'].unique():
        x_test[i] = x_test[i].map(low_vals).fillna(inf_const).astype(int)
    data[agg_num]['y_test'] = x_test.copy()
    


In [239]:
df_y_submit = []
for n, agg_num in enumerate(agg_l):
    df_y = pd.DataFrame(columns = data[agg_num]['y_proc_columns'], index = data[agg_num]['x_test_submit'].index)
    for tm in list(y_vals[agg_num][y_vals[agg_num]['num_of_downtimes']!=0].index):
        df_y[tm] = data[agg_num]['y_test'][tm]

    df_y = df_y.fillna(inf_const).astype(int)
    df_y.columns = data[agg_num]['y_raw_columns']
    
    df_y_submit.append(df_y)
df_y_submit = pd.concat(df_y_submit, axis=1)

In [243]:
df_th = pd.Series(threshs).reset_index()
df_th.columns = ['agg_num', 'time_period', 'tm', 'val']
df_th.to_csv('df_th.csv', index=False)

In [247]:
df_y_submit.to_parquet('task_3.parquet')

## Task 1

In [396]:
expeted_v = int(len(df_int)/2)
df_int_tmp = df_int.copy()
ex_v = downtime_cols.sum(axis=0)[agg_l]
ex_v = (ex_v*expeted_v/ex_v.sum()).round().astype(int)

In [397]:
for agg_num in agg_l:
    df_int_tmp[agg_num] = None
    for n, i in df_int.iterrows():
        score = data[agg_num]['x_test_submit'].loc[i['start']:i['finish'], 'anomaly_score_2h_quantile_stats']
        df_int_tmp.loc[n, agg_num] = score.mean()
    df_int_tmp[agg_num] = df_int_tmp[agg_num].fillna(0).astype(float)
df_int_tmp[5] = df_int_tmp[5]*1.1
df_int_tmp[4] = df_int_tmp[4]*1.1

In [398]:
df_int_tmp['max_score'] = df_int_tmp[agg_l].max(axis=1)
dict_most_freq_tm = downtime_cols.T.idxmax(axis=1).to_dict()
df_int_tmp['machine_tmp'] = df_int_tmp[agg_l].idxmax(axis=1)
df_int_tmp['tm_tmp'] = df_int_tmp['machine_tmp'].map(dict_most_freq_tm)
df_int_tmp['machine_tmp_str'] = 'ЭКСГАУСТЕР А/М №' + df_int_tmp['machine_tmp'].astype(str)

In [399]:
for agg_num in agg_l:
    dict_ = {}
    for i in range(len(data[agg_num]['y_proc_columns'])):
        dict_[data[agg_num]['y_proc_columns'][i]] = data[agg_num]['y_raw_columns'][i]
    df_int_tmp.loc[df_int_tmp['machine_tmp']==agg_num, 'tm_tmp'] = \
            df_int_tmp.loc[df_int_tmp['machine_tmp']==agg_num, 'tm_tmp'].map(dict_)

In [400]:
for agg_num in agg_l:
    indexes = df_int_tmp.loc[df_int_tmp['machine_tmp']==agg_num].sort_values('max_score')[::-1].head(ex_v[agg_num]).index
    df_int_tmp.loc[indexes, 'machine'] = df_int_tmp.loc[indexes, 'machine_tmp_str']
    df_int_tmp.loc[indexes, 'tm'] = df_int_tmp.loc[indexes, 'tm_tmp']

In [401]:
df_int_tmp[['start', 'finish', 'machine', 'tm']].to_excel('task_1.xlsx')

In [415]:
means.reset_index().rename(columns = {'index':'col', 0:'v'}).to_csv('means.csv', index=False)
std.reset_index().rename(columns = {'index':'col', 0:'v'}).to_csv('std.csv', index=False)
df_th.to_csv('threshs.csv', index=False)

In [ ]:
397.8801212506967 + 2*1139.0966411502727